In [2]:
import spacy
import random
import json
import pandas as pd
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Training Data: Intent classification dataset with more examples
training_data = [
    ("Who won the last F1 race?", "race_result"),
    ("Who was the last F1 race winner?", "race_result"),
    ("Give me the latest Formula 1 standings", "standings"),
    ("What are the current F1 standings?", "standings"),
    ("Tell me about Lewis Hamilton", "driver_info"),
    ("What do you know about Lewis Hamilton?", "driver_info"),
    ("Who is the GOAT in F1?", "goat"),
    ("Who is the greatest F1 driver of all time?", "goat"),
    ("Compare Ferrari and Mercedes cars", "car_comparison"),
    ("How do Ferrari and Mercedes cars compare?", "car_comparison"),
    ("Which is the fastest road car?", "car_speed"),
    ("What is the fastest car in the world?", "car_speed"),
    ("List top F1 teams", "teams"),
    ("Which are the best F1 teams?", "teams"),
    ("Explain aerodynamics in racing", "aerodynamics"),
    ("What is aerodynamics in F1?", "aerodynamics"),
    ("Hello!", "greeting"),
    ("Hi there!", "greeting"),
    ("Hey!", "greeting"),
    ("How are you?", "greeting"),
    ("What is the best F1 engine?", "engine_info"),
    ("Which F1 team has the best engine?", "engine_info"),
    ("Who has the most F1 championships?", "championships"),
    ("Which driver has won the most F1 championships?", "championships"),
    ("Tell me about Red Bull Racing", "team_info"),
    ("What do you know about Red Bull Racing?", "team_info"),
    ("What is DRS in Formula 1?", "drs"),
    ("Explain DRS in F1", "drs"),
    ("Best biryani in the world?", "biryani"),
]

# Convert to DataFrame
df = pd.DataFrame(training_data, columns=["text", "intent"])

# Train Intent Classifier
vectorizer = TfidfVectorizer()
classifier = LogisticRegression()
pipeline = make_pipeline(vectorizer, classifier)
pipeline.fit(df["text"], df["intent"])

# Predefined responses
responses = {
    "race_result": "Max Verstappen won the last F1 race! 🏆",
    "standings": "Check the latest F1 standings at the official F1 website!",
    "driver_info": "Lewis Hamilton is a 7-time world champion!",
    "goat": "LEWIS HAMILTON!!!!",
    "car_comparison": "Ferrari cars are known for speed, while Mercedes focuses on aerodynamics.",
    "car_speed": "The Bugatti Chiron Super Sport 300+ is the fastest road car!",
    "teams": "Top F1 teams: Mercedes, Red Bull, Ferrari, McLaren...",
    "aerodynamics": "Aerodynamics in racing helps reduce drag and increase downforce for better handling.",
    "greeting": "Hello! How can I assist you with Formula 1 or car-related queries today?",
    "engine_info": "Mercedes-AMG and Honda have been dominant in F1 engines.",
    "championships": "Michael Schumacher and Lewis Hamilton both have 7 F1 championships.",
    "team_info": "Red Bull Racing is a dominant F1 team known for its aggressive strategy and fast pit stops.",
    "drs": "DRS (Drag Reduction System) helps F1 cars reduce drag and increase straight-line speed.",
    "biryani": "Hyderabadi dum biryani is the best biryani in the world! 🍛",
}

# Fetch real-time F1 news from a web source
def get_latest_f1_news():
    try:
        response = requests.get("https://ergast.com/api/f1/current/last/results.json")
        if response.status_code == 200:
            data = response.json()
            last_race = data["MRData"]["RaceTable"]["Races"][0]
            winner = last_race["Results"][0]["Driver"]
            return f"The latest F1 race was won by {winner['givenName']} {winner['familyName']}!"
        else:
            return "I couldn't fetch the latest F1 race results right now. Try again later."
    except Exception:
        return "Error retrieving F1 data. Please check back later."

# Terminal-based chatbot
def chat():
    print("Formula 1 & Car Chatbot! Type 'exit' to stop.")
    while True:
        user_input = input("You: ").strip()
        print(f"Question asked: {user_input}")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        try:
            intent = pipeline.predict([user_input])[0]
            response = responses.get(intent, "I'm not sure about that. I can help with Formula 1 or car-related questions!")
            if intent == "race_result":
                response = get_latest_f1_news()
        except Exception:
            response = "I'm not sure about that. I can help with Formula 1 or car-related questions!"
        print("Bot:", response)

if __name__ == "__main__":
    chat()


Formula 1 & Car Chatbot! Type 'exit' to stop.
Question asked: Best biryani in the world?
Bot: Hello! How can I assist you with Formula 1 or car-related queries today?
Question asked: Best biryani in the world?
Bot: Hello! How can I assist you with Formula 1 or car-related queries today?
Question asked: exit
Goodbye!
